In [1]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from tqdm import tqdm

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set paths and parameters
BASE_DIR = r"A:\Anomaly Detection\Data"  # Replace with your dataset path
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 1e-4

# Custom Dataset Class
class MVTecDataset(Dataset):
    def __init__(self, base_dir, category, phase="train", transform=None):
        self.base_dir = base_dir
        self.category = category
        self.phase = phase
        self.transform = transform
        self.image_paths = []
        self.load_dataset()

    def load_dataset(self):
        img_dir = os.path.join(self.base_dir, self.category, self.phase)
        for root, _, files in os.walk(img_dir):
            for file in files:
                if file.endswith('.png'):
                    self.image_paths.append(os.path.join(root, file))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        return image

# Data Transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
])

# Data Loaders
def get_dataloader(category, phase):
    dataset = MVTecDataset(BASE_DIR, category, phase, transform)
    return DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=(phase=="train"))

# Autoencoder Model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = models.efficientnet_b0(pretrained=True)
        self.encoder.classifier = nn.Identity()  # Remove the classification layer
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1280, 512, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder.features(x)
        x = self.decoder(x)
        return x

# Training Function
def train_model(model, dataloader, criterion, optimizer, epochs):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            images = images.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, images)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        epoch_loss = running_loss / len(dataloader.dataset)
        print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")

# Sequential Training on All Categories in Dataset
def train_on_all_categories(base_dir):
    categories = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    model = Autoencoder()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    for category in categories:
        print(f"Training on category: {category}")
        train_loader = get_dataloader(category, "train")

        # If a previously trained model exists, load it
        model_path = f"{category}_autoencoder.pth"
        if os.path.exists(model_path):
            print(f"Loading pre-trained model for category: {category}")
            model.load_state_dict(torch.load(model_path))

        # Train the model
        train_model(model, train_loader, criterion, optimizer, EPOCHS)

        # Save the model
        torch.save(model.state_dict(), model_path)
        print(f"Model saved for category: {category}")

# Main Execution
if __name__ == "__main__":
    train_on_all_categories(BASE_DIR)


Using device: cuda


c:\Users\ASUS\anaconda3\envs\NewGPUTorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\anaconda3\envs\NewGPUTorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training on category: bottle


Epoch 1/10: 100%|██████████| 14/14 [00:07<00:00,  1.76it/s]


Epoch 1 Loss: 0.1255


...

Epoch 8 Loss: 0.0024


Epoch 9/10: 100%|██████████| 15/15 [00:07<00:00,  1.88it/s]


Epoch 9 Loss: 0.0024


Epoch 10/10: 100%|██████████| 15/15 [00:08<00:00,  1.81it/s]


Epoch 10 Loss: 0.0021
Model saved for category: zipper


In [3]:
def classify_image(input_image_path, base_dir):
    categories = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
        transforms.ToTensor(),
    ])

    input_image = cv2.imread(input_image_path)
    input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    input_image = transform(input_image).unsqueeze(0).to(device)

    model = Autoencoder().to(device)
    defect_detected = False
    defect_info = {}

    for category in categories:
        model_path = f"{category}_autoencoder.pth"
        mask_dir = os.path.join(base_dir, category, "ground_truth")

        if not os.path.exists(model_path):
            print(f"Model for category {category} not found.")
            continue

        model.load_state_dict(torch.load(model_path))
        model.eval()

        with torch.no_grad():
            reconstructed = model(input_image)
            loss = nn.functional.mse_loss(reconstructed, input_image).item()

        print(f"Reconstruction loss for category {category}: {loss:.4f}")

        if loss > 0.05:  # Threshold for anomaly detection
            defect_detected = True
            defect_info[category] = {
                "loss": loss,
                "mask_folder": mask_dir
            }

    if defect_detected:
        print("Defects detected in the following categories:")
        for category, info in defect_info.items():
            print(f"Category: {category}, Loss: {info['loss']:.4f}, Mask Folder: {info['mask_folder']}")
    else:
        print("No defects detected.")

    return defect_info if defect_detected else "normal"



In [4]:
test_image_path = r"A:\Anomaly Detection\Data\bottle\test\broken_small\000.png"  # Replace with the path to your test image
result = classify_image(test_image_path, BASE_DIR)
print(f"Final classification result: {result}")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14792\796808869.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Reconstruction loss for category bottle: 0.0031
Reconstruction loss for category cable: 0.1641
Reconstruction loss for category capsule: 0.3422
Reconstruction loss for category carpet: 0.4111
Reconstruction loss for category grid: 0.4185
Reconstruction loss for category hazelnut: 0.2813
Reconstruction loss for category leather: 0.4182
Reconstruction loss for category metal_nut: 0.4143
Reconstruction loss for category pill: 0.4185
Reconstruction loss for category screw: 0.2610
Reconstruction loss for category tile: 0.4144
Reconstruction loss for category toothbrush: 0.4037
Reconstruction loss for category transistor: 0.2818
Reconstruction loss for category wood: 0.2397
Reconstruction loss for category zipper: 0.3919
Defects detected in the following categories:
Category: cable, Loss: 0.1641, Mask Folder: A:\Anomaly Detection\Data\cable\ground_truth
Category: capsule, Loss: 0.3422, Mask Folder: A:\Anomaly Detection\Data\capsule\ground_truth
Category: carpet, Loss: 0.4111, Mask Folder: A: